# LangChain Mega-Tutorial (Hands‑On, End‑to‑End)

> Generated on 2025-09-22 05:36 — drop this into your environment and run cells top‑to‑bottom.

This notebook is a **very descriptive, comprehensive LangChain tutorial** covering *most practical topics* end‑to‑end:
- Installation & environment setup (Python, pip/uv, API keys)
- Core concepts: **LLMs, ChatModels, Prompts, Output Parsers**
- **LCEL (LangChain Expression Language)** & Runnables
- **Chains** (sequential, map-reduce), **Routers**
- **Tools** & **Function/Tool Calling**
- **Agents** (ReAct, OpenAI Tools style), **Toolkits**
- **Document Loaders**, **Text Splitters**, **Embeddings**, **VectorStores**
- **Retrievers** & **RAG** (basic → advanced), **Multi‑query**, **HyDE**, **Re‑ranking**
- **Chat History & Memory** (buffer, summary, entity, vector)
- **Callbacks, Tracing, Streaming, Async & Batching**
- **Structured Output** (Pydantic/JSON), **Guardrails / Safety hooks**
- **Evaluation** (unit tests for prompts/chains, Ragas‑style ideas)
- **Caching & Persistence**, **Checkpointers**
- **LangGraph** basics for stateful graphs (agents & RAG workflows)
- **Deployment**: **LangServe** quick start, FastAPI microservice pattern
- **Observability**: **LangSmith** tracing + dataset evals (optional)
- Tips, pitfalls, and production checklists

> 📝 **Note:** Many sections use optional providers (OpenAI, Anthropic, Groq, etc.). You can run with any supported model by swapping the model and client bits. Cells are written to be **provider‑agnostic** where possible.

## 0) Prerequisites & Environment

Run this cell to install the packages you need. If you’re offline, skip now and run later in your own environment.

In [ ]:
# If you're using a fresh environment, uncomment and run:
# !pip install -U "langchain>=0.2.16" "langchain-community>=0.2.11" "langchain-openai>=0.2.3" #                "faiss-cpu>=1.8.0" "pydantic>=2.6.0" "tiktoken" "python-dotenv" "fastapi" "uvicorn" #                "langchainhub" "beautifulsoup4" "pypdf" "unstructured" "lxml" "httpx" "aiohttp"
#
# Optional (choose your provider integrations as needed):
# !pip install -U "openai>=1.44.0" "anthropic>=0.34.0" "groq>=0.9.0" "cohere>=5.5.8"
# !pip install -U "ragas" "chromadb" "rank-bm25" "sentence-transformers"
#
# For LangGraph/LangServe/LangSmith (optional):
# !pip install -U "langgraph>=0.2.38" "langserve>=0.3.0" "langsmith>=0.1.129"

### Environment Variables

Create a `.env` file or export env vars before running provider cells:
```bash
export OPENAI_API_KEY="..."         # or ANTHROPIC_API_KEY / GROQ_API_KEY / COHERE_API_KEY
export LANGCHAIN_TRACING_V2="true"  # optional (LangSmith)
export LANGCHAIN_API_KEY="..."      # optional (LangSmith)
```

## 1) Hello, LangChain — Minimal LLM & ChatModel

We demonstrate both text LLMs and chat models using the **LangChain** wrappers. Swap `OpenAI` with your provider of choice.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# Minimal: chat completion
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)  # swap to your model
resp = chat.invoke([HumanMessage(content="In one sentence, explain LangChain.")] )
print(resp.content)

## 2) Prompt Templates & Output Parsers

**PromptTemplate** lets you parameterize text; **ChatPromptTemplate** structures multi‑turn prompts.  
**Output parsers** help convert responses into Pythonic types (JSON, lists, Pydantic models).

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Idea(BaseModel):
    title: str = Field(..., description="Short idea title")
    rationale: str

parser = PydanticOutputParser(pydantic_object=Idea)

prompt = PromptTemplate(
    template=(
        "Generate a startup idea about {topic}."
        "{format_instructions}"
    ),
    input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.4) | parser
result = chain.invoke({"topic": "personal finance for GenZ"})
result

## 3) LCEL (LangChain Expression Language) & Runnables

LCEL composes components with the `|` operator. Everything becomes a **Runnable**; you can **invoke**, **batch**, and **stream**.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise assistant."),
    ("human", "Summarize in 15 words: {text}")
])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
to_upper = RunnableLambda(lambda s: s.upper())

lcel_chain = prompt | model | StrOutputParser() | to_upper
lcel_chain.invoke({"text": "LangChain helps you build LLM apps by composing components cleanly."})

## 4) Chains: Sequential, Map‑Reduce, Router

Chains coordinate multiple steps. Below: a simple sequential chain, then a toy map‑reduce summarizer.

In [ ]:
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

docs = [Document(page_content=s) for s in [
    "LangChain is a framework to build with LLMs.",
    "It offers integrations for prompts, models, memory, tools, and more.",
    "LCEL composes components using a pipe operator."
]]

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
map_reduce: BaseCombineDocumentsChain = load_summarize_chain(model, chain_type="map_reduce")
map_reduce.run(docs)

## 5) Tools & Function/Tool Calling

Expose Python functions as **tools**; modern models can decide when to call them.

In [ ]:
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate

@tool
def get_weather(city: str) -> str:
    """Return sunny/foggy/rainy (toy)."""
    return f"Weather in {city}: sunny 27°C (demo)"

tools = [get_weather]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = create_openai_tools_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

executor.invoke({"input": "What is the weather in Jaipur? Should I carry sunglasses?"})

## 6) Agents: ReAct & OpenAI Tools Style

**Agents** reason about which tools to use and in what order. Use **ReAct** for traceable reasoning or **Tools style** for built‑in function calling.

In [ ]:
# ReAct-style agent example (uses zero-shot-react-description)
from langchain.agents import load_tools, initialize_agent, AgentType

search_tools = []  # e.g., load_tools(["ddg-search"]) if installed & configured
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

react_agent = initialize_agent(
    tools + search_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
react_agent.run("Plan a 1-day food crawl in Lucknow. Use get_weather if helpful.")

## 7) Documents, Loaders & Text Splitters

Load data from the web, PDFs, or files; split into chunks for embeddings.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.langchain.com/")  # try on any public page
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
len(chunks), chunks[0].page_content[:200]

## 8) Embeddings & VectorStores (FAISS)

Create embeddings for chunks and build a FAISS index.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(chunks[:20], embedding=emb)  # index a subset for speed
retriever = vs.as_retriever(search_kwargs={"k": 4})
retriever.get_relevant_documents("What is LangChain for?")[0].page_content[:300]

## 9) RAG: Basic Retriever‑Augmented Generation

Classic **RAG** = Retrieve top‑k docs → stuff into prompt → generate answer.

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    chain_type="stuff",
    retriever=retriever
)
qa.invoke({"query": "Summarize the purpose of LangChain and LCEL."})

## 10) RAG: Advanced Techniques (Multi‑Query, HyDE, Re‑ranking)

- **Multi‑Query**: generate diverse queries to improve recall  
- **HyDE**: create a hypothetical answer and embed it to retrieve  
- **Re‑ranking**: score retrieved docs, keep the best

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

mqr = MultiQueryRetriever.from_llm(retriever=vs.as_retriever(), llm=ChatOpenAI(model="gpt-4o-mini"))
docs = mqr.get_relevant_documents("How does LCEL help composition?")
[ (i, d.metadata.get("source",""), d.page_content[:100]) for i, d in enumerate(docs) ]

## 11) Conversation Memory (Buffer, Summary, Entity, Vector)

Use memory to maintain context across turns.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conv = ConversationChain(llm=llm, memory=memory, verbose=True)

conv.predict(input="Hi, I'm planning a trip to Kerala.")
conv.predict(input="I love beaches and budget is medium.")
conv.predict(input="Remind me: what did I say I like?")

## 12) Callbacks, Tracing, Streaming & Async/Batching

Callbacks provide hooks (start/end, token events). Enable **LangSmith** for tracing. Streaming yields tokens incrementally.

In [ ]:
import asyncio
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

streaming_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
streaming_llm.invoke("Stream a haiku about monsoons in Pune.")

# Async batching demo
async def a_batch(prompts):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    results = await llm.abatch(prompts)
    return [r.content for r in results]

asyncio.run(a_batch(["One fact about mangoes.", "One fact about coconuts."]))

## 13) Structured Output & Guardrails

Use Pydantic schemas for reliable JSON. Add simple content filters/validators as pre/post hooks.

In [ ]:
from pydantic import BaseModel, field_validator
from langchain.schema.runnable import RunnablePassthrough

class Place(BaseModel):
    name: str
    city: str
    budget: float
    @field_validator("budget")
    def non_negative(cls, v):
        if v < 0: raise ValueError("Budget must be non-negative")
        return v

parser = PydanticOutputParser(pydantic_object=Place)

system_guard = RunnablePassthrough()  # stub; insert moderation calls here if you have them
prompt = PromptTemplate(
    template="Recommend a venue as JSON.{fmt} User: {wish}",
    input_variables=["wish"],
    partial_variables={"fmt": parser.get_format_instructions()}
)
chain = system_guard | (prompt | ChatOpenAI(model="gpt-4o-mini") | parser)
chain.invoke({"wish": "I need a party place in Jaipur under $200"})

## 14) Evaluation Basics

Write quick tests for prompts/chains. For larger evals use **LangSmith datasets** or **Ragas**.

In [ ]:
def assert_non_empty(s: str):
    assert isinstance(s, str) and len(s.strip()) > 0, "Empty output"

out = (ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()).invoke("Return a single word: hello")
assert_non_empty(out)
print("✅ Simple eval passed.")

## 15) Caching & Persistence

Cache LLM calls (in‑memory, SQLite, Redis, etc.) to speed up development and cut costs.

In [ ]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(llm.invoke("Say 'cached hello'").content)  # first
print(llm.invoke("Say 'cached hello'").content)  # served from cache (if enabled/provider supports)

## 16) LangGraph (Optional): Stateful Graphs for Agents & RAG

Use **LangGraph** to build deterministic graphs around LLM steps, tools, and state. Below is a tiny graph with two nodes.

In [ ]:
# Minimal LangGraph example (ensure langgraph is installed)
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    message: str

def node_upper(state: State):
    return {"message": state["message"].upper()}

def node_suffix(state: State):
    return {"message": state["message"] + " ✅"}

graph = StateGraph(State)
graph.add_node("upper", node_upper)
graph.add_node("suffix", node_suffix)
graph.set_entry_point("upper")
graph.add_edge("upper", "suffix")
graph.add_edge("suffix", END)

app = graph.compile()
app.invoke({"message": "hello graph"})

## 17) Deployment with LangServe (Optional)

Expose your chain with **LangServe**. Start FastAPI and call your chain over HTTP.

In [ ]:
# main.py (example)
# from fastapi import FastAPI
# from langserve import add_routes
# from langchain.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
#
# app = FastAPI()
# chain = ChatPromptTemplate.from_messages([("human", "{input}")]) | ChatOpenAI(model="gpt-4o-mini")
# add_routes(app, chain, path="/chain")
#
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=8000)
#
# Run server:
# uvicorn main:app --reload --port 8000
#
# Then call:
# curl -X POST http://localhost:8000/chain/invoke -H "Content-Type: application/json" -d '{"input": {"input":"Hello"}}'

## 18) LangSmith Observability (Optional)

Turn on tracing and link runs to datasets and evals.

In [ ]:
# export LANGCHAIN_TRACING_V2=true
# export LANGCHAIN_API_KEY=...
#
# from langsmith import Client
# client = Client()
# dataset = client.create_dataset("my-eval-set", description="Toy Q/A")
# client.create_example(inputs={"q":"What is LCEL?"}, outputs={"a":"..."}, dataset_id=dataset.id)
# # Run your chain across dataset via LangSmith UI or SDK.

## 19) Production Checklist & Pitfalls

- **Determinism**: fix temperatures, seed where supported; add unit tests for prompts.
- **Resilience**: timeouts, retries, fallbacks (secondary model or cached answer).
- **Guardrails**: input validation, output schemas, allow/deny lists, PII scrubbing.
- **Observability**: tracing, logs, metrics; capture prompts+latency+cost.
- **Cost control**: batching, compression/summarization, smaller contexts.
- **Data**: deduplicate, chunk sensibly, consider re‑ranking and hybrid search.
- **Ops**: version datasets, prompts, chains; blue/green deploys; rollbacks.
- **Security**: never log secrets; sign tool calls; sandbox untrusted code.

## 20) Appendix: Swapping Providers Quickly

Replace `ChatOpenAI` with e.g. `from langchain_groq import ChatGroq` or `from langchain_anthropic import ChatAnthropic`, update env vars, and keep the rest of the pipeline identical.